In [1]:
from IPython.display import Image, display

import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv

load_dotenv()

config_list = [
    {
        'model': 'gpt-4o',
        'api_key':  os.getenv("OPENAI_API_KEY"),
        'tags': ['tool', 'gpt-4'],
    },
    {
        'model': 'gemini-1.5-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.5-flash',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.0-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['gemini'],
    }
]

import json
config_list_file_name = ".config_list"
with open(config_list_file_name, "w") as file:
    json.dump(config_list, file, indent=4)

In [5]:

import time
import markdown2
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from autogen_core.tools import FunctionTool

from reportlab.lib.pagesizes import letter

def generate_pdf(content: str, filename: str = "output.pdf", output_dir: str = "output") -> str:
    """Generate a PDF file from text content with proper word wrapping and markdown support."""
    
    os.makedirs(output_dir, exist_ok=True)

    # Ensure the filename has a .pdf extension
    if not filename.lower().endswith(".pdf"):
        filename += ".pdf"

    filepath = os.path.join(output_dir, filename)

    # Convert Markdown to HTML
    html_content = markdown2.markdown(content)

    # Create a PDF document
    doc = SimpleDocTemplate(filepath, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # Convert HTML to ReportLab Paragraph
    for paragraph in html_content.split("\n"):
        if paragraph.strip():  # Ignore empty lines
            story.append(Paragraph(paragraph, styles["Normal"]))
            story.append(Spacer(1, 12))  # Add spacing between paragraphs

    # Build the PDF
    doc.build(story)

    return filepath

def google_search(query: str, num_results: int = 2, max_chars: int = 500, max_attempts: int = 2) -> list:
    """Search Google for patents, keywords and return results with a stopping condition."""
    
    api_key = os.getenv("GOOGLE_API_KEY")
    search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")

    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    
    attempts = 0
    results = []

    while attempts < max_attempts and len(results) < num_results:
        params = {"key": api_key, "cx": search_engine_id, "q": query, "num": num_results}
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(response.json())
            raise Exception(f"Error in API request: {response.status_code}")

        new_results = response.json().get("items", [])
        results.extend(new_results[: num_results - len(results)])

        if len(results) >= num_results:
            break  # Stop searching if enough results are found

        attempts += 1
        time.sleep(1)  # Be respectful to API rate limits

    return results



def arxiv_search(query: str, max_results: int = 2, max_attempts: int = 2) -> list:
    """Search Arxiv for papers and return results with a stopping condition."""
    import arxiv

    client = arxiv.Client()
    
    attempts = 0
    results = []

    while attempts < max_attempts and len(results) < max_results:
        search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

        for paper in client.results(search):
            if len(results) >= max_results:
                break
            results.append({
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            })

        if len(results) >= max_results:
            break  # Stop searching if enough results are found
        
        attempts += 1
        time.sleep(1)  # Avoid overloading ArXiv API

    return results

google_search_tool = FunctionTool(
    google_search,
    name="google_search",
    description="Search Google for information, returning results with snippets and body content."
)

arxiv_search_tool = FunctionTool(
    arxiv_search,
    name="arxiv_search",
    description="Search Arxiv for publications related to a given topic, including abstracts citation and pdf links."
)

pdf_generator_tool = FunctionTool(
    generate_pdf,
    name="PDF_Generator",
    description="Generate a PDF document from the content generated by Researcher and save it in the output folder with markdown support."
)

In [6]:
gpt_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gpt-4"]})
gpt_llm_config = {"config_list": gpt_config_list, "timeout": 120}

gemini_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})
gemini_llm_config = {"config_list": gemini_config_list, "timeout": 120}

user_proxy = autogen.UserProxyAgent(
    name="User_Proxy_Agent",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": True,
        # "use_docker": False,
        # "executor": LocalCommandLineCodeExecutor(work_dir="working_dir"),
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    # human_input_mode="TERMINATE",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)


coder = autogen.AssistantAgent(
    name="Expert_Coder",
    system_message=(
        "You are responsible for implementing the necessary Python functions. "
        "You are also responsible for executing the required functions, including, debugging, "
        "searching, installing tools, or running scripts. If you complete all of your instructed tasks, "
        "report back to the 'User_Proxy_Agent' or 'Strategic_Planner' with the results."
    ),
    
    llm_config=gemini_llm_config,
    function_map={"arxiv_search": arxiv_search, "generate_pdf": generate_pdf},
)

# coder.register_tool(name="arxiv_search", description="arxiv_search for searching publications in arxiv")(arxiv_search)
# coder.register_tool(name="pdf_generator_tool", description="used for generating pdf from text")(pdf_generator_tool)

planner = autogen.AssistantAgent(
    name="Strategic_Planner",
    system_message=(
        "You are responsible for planning. Once the plan is made, "
        "explicitly instruct 'Expert_Coder' to perform coding-related or execution-related tasks, "
        "when final task is done, back to 'User_Proxy_Agent' with TERMINATE, otherwise proceed the next step"
    ),
    llm_config=gemini_llm_config,
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, coder, planner], 
    messages=[], 
    max_round=20,
    allow_repeat_speaker = False,
    max_retries_for_selecting_speaker=3,
    )
# manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt_llm_config)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gemini_llm_config)
user_proxy.initiate_chat(
    manager,
    message="Do complete the following tasks: "
    "1. Find top cited 10 latest publications about the topic, Advance ANC feature for Earbuds and AirPods on arxiv within year 2023-2025. "
    "2. Prepare a detail technical review document in PDF format. You must include references of the publications where required. "
    "3. Save the generated document in output/anc_doc.pdf file. If you cannot generate PDF you can use required tools to achieve the goal. ",
    summary_method="reflection_with_llm",
)

AttributeError: 'AssistantAgent' object has no attribute 'register_tool'